In [1]:
from pyspark.sql import SparkSession
import os
from pyspark.sql import *

# Prepare Cassandra data structures to store the results.

In [2]:
from cassandra.cluster import Cluster
cluster = Cluster()
session = cluster.connect('rohin')

In [5]:
'''prepare statement and create table cassandra'''


statement = session.prepare("create table rohin.vechicle_group_M50 (cosit int, classname text, count int, stream_id int, primary key (cosit, classname, stream_id))")
session.execute(statement)

statement = session.prepare("create table rohin.Average_velocity_M50 (cosit int, avg_speed float, stream_id int, primary key (cosit, stream_id))")
session.execute(statement)

statement = session.prepare("create table rohin.Bussiest_Nodes_m50 (cosit int, count int, stream_id int, primary key (cosit,stream_id))")
session.execute(statement)

statement = session.prepare("create table rohin.HGV_traffic_M50 (classname text, count int, stream_id int, primary key (classname,stream_id))"  )
session.execute(statement)




In [4]:
''' drop all tables created above'''
session.execute("drop table if exists rohin.vechicle_group_M50")
session.execute("drop table if exists rohin.Average_velocity_M50")
session.execute("drop table if exists rohin.Bussiest_Nodes_m50")
session.execute("drop table if exists rohin.HGV_traffic_M50")

# Insert Statement Processing

In [6]:
def insert_Q1(cosit,classname,count,stream_id):
    '''prepare statment and execute in cassandra'''
    statement = session.prepare("insert into rohin.vechicle_group_M50 (cosit, classname, count, stream_id) values (?,?,?,?)")
    session.execute(statement, (cosit,classname,count,stream_id))
def insert_Q2(cosit,avg_speed,stream_id):
    statement = session.prepare("insert into rohin.Average_velocity_M50 (cosit, avg_speed, stream_id) values (?,?,?)")
    session.execute(statement, (cosit,avg_speed,stream_id))
def insert_Q3(cosit,count,stream_id):
    statement = session.prepare("insert into rohin.Bussiest_Nodes_m50 (cosit, count, stream_id) values (?,?,?)")
    session.execute(statement, (cosit,count,stream_id))
def insert_Q4(classname,count,stream_id):
    statement = session.prepare("insert into rohin.HGV_traffic_M50 (classname, count, stream_id) values (?,?,?)")
    session.execute(statement, (classname,count,stream_id))

In [7]:
# select * from vechicle_group_M50
# select * from Average_velocity_M50
# select * from Bussiest_Nodes_m50
# select * from HGV_traffic_M50

# Dump Cassandra

In [9]:
spark = SparkSession.builder.appName('Read CSV File into DataFrame').getOrCreate()
stream_ID = 0
import time
while(True):
    path = 'Counter/countdata'+str(stream_ID)+'.csv'
    print(path)
    if os.path.isfile(path):
        Stream_lit_data = spark.read.csv(path, header=True, inferSchema=True)
        M50_nodes = [1012,1014,1500,1501,1502,1503,1504,1505,1506,1507,1508,1509,15010,15011,15012]
        M50Datapoints = Stream_lit_data.filter(Stream_lit_data.cosit.isin(M50_nodes))
        
        vechicle_group_M50 = M50Datapoints.groupby('cosit','classname').count().withColumnRenamed('count','Vehicle_Frequency').withColumnRenamed('cosit','cosit_id').withColumnRenamed('classname','classname_id')
        
        for row in vechicle_group_M50.collect():
            insert_Q1(row[0],row[1],row[2],stream_ID)
            
    
        Average_velocity_M50 = M50Datapoints.groupby('cosit').agg({'speed':"avg"}).withColumnRenamed('avg(speed)','Mean_Velocity').withColumnRenamed('cosit','Node_ID')
        
        for row in Average_velocity_M50.collect():
            insert_Q2(row[0],row[1],stream_ID)
            
            
        Bussiest_Nodes_m50 = M50Datapoints.groupBy('cosit').count().sort('cosit',ascending=False).withColumnRenamed('count','Vehicle_Frequency').withColumnRenamed('cosit','Node_ID')
        
        for row in Bussiest_Nodes_m50.collect()[:3]:
            insert_Q3(row[0],row[1],stream_ID)
        print(len(Bussiest_Nodes_m50.collect()))  
            
        HGV_list = ['HGV_ART','HGV_RIG']
        HGV_traffic_M50 = M50Datapoints.filter(M50Datapoints['classname'].isin(HGV_list)).groupby('classname').count()
        
        for row in HGV_traffic_M50.collect():
            insert_Q4(row[0],row[1],stream_ID)
            
            
        print('Start    '+str(time.strftime("%H:%M:%S")))
        time.sleep(5)
        print('End  '+str(time.strftime("%H:%M:%S")))
        stream_ID += 1
    else:
        break

C:\Python27\lib\site-packages\pyspark\context.py:227: DeprecationWarning: Support for Python 2 and Python 3 prior to version 3.6 is deprecated as of Spark 3.0. See also the plan for dropping Python 2 support at https://spark.apache.org/news/plan-for-dropping-python-2-support.html.
  DeprecationWarning)


Counter/countdata0.csv
0
Start    01:59:13
End  01:59:18
Counter/countdata1.csv
1
Start    01:59:26
End  01:59:31
Counter/countdata2.csv
3
Start    01:59:42
End  01:59:47
Counter/countdata3.csv
2
Start    01:59:55
End  02:00:00
Counter/countdata4.csv
0
Start    02:00:04
End  02:00:09
Counter/countdata5.csv
2
Start    02:00:15
End  02:00:20
Counter/countdata6.csv
2
Start    02:00:25
End  02:00:30
Counter/countdata7.csv
3
Start    02:00:36
End  02:00:41
Counter/countdata8.csv
1
Start    02:00:46
End  02:00:51
Counter/countdata9.csv
3
Start    02:00:55
End  02:01:00
Counter/countdata10.csv
1
Start    02:01:05
End  02:01:10
Counter/countdata11.csv
2
Start    02:01:15
End  02:01:20
Counter/countdata12.csv
4
Start    02:01:29
End  02:01:34
Counter/countdata13.csv
3
Start    02:01:39
End  02:01:44
Counter/countdata14.csv
0
Start    02:01:48
End  02:01:53
Counter/countdata15.csv
4
Start    02:01:59
End  02:02:04
Counter/countdata16.csv
3
Start    02:02:14
End  02:02:19
Counter/countdata17.csv
